In [ ]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer,
    BertModel,
)
import json

In [9]:
class MultiLabelBERT_Political(torch.nn.Module):
    """BERT model for multi-label token classification."""

    def __init__(
        self,
        bert_model_name,
        num_stance_labels,
        num_group_appeals_labels,
        num_endorsement_labels,
    ):
        super(MultiLabelBERT_Political, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.stance_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_stance_labels
        )
        self.group_appeals_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_group_appeals_labels
        )
        self.endorsement_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_endorsement_labels
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = (
            outputs.last_hidden_state
        )  # (batch_size, max_len, hidden_size)

        stance_logits = self.stance_classifier(sequence_output)
        group_appeals_logits = self.group_appeals_classifier(sequence_output)
        endorsement_logits = self.endorsement_classifier(sequence_output)

        return stance_logits, group_appeals_logits, endorsement_logits

class MultiLabelBERT_Sentiment(torch.nn.Module):
    """BERT model for multi-label token classification."""

    def __init__(self, bert_model_name, num_hyperbol_labels, num_sentiment_labels):
        super(MultiLabelBERT_Sentiment, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.hyperbole_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_hyperbol_labels
        )
        self.sentiment_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_sentiment_labels
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = (
            outputs.last_hidden_state
        )  # (batch_size, max_len, hidden_size)

        hyperbole_logits = self.hyperbole_classifier(sequence_output)
        sentiment_logits = self.sentiment_classifier(sequence_output)

        return hyperbole_logits, sentiment_logits

class MultiLabelBERT_Readability(torch.nn.Module):
    """BERT model for multi-label token classification."""

    def __init__(self, bert_model_name, num_vagueness_labels, num_readability_labels):
        super(MultiLabelBERT_Readability, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.vagueness_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_vagueness_labels
        )
        self.readability_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_readability_labels
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = (
            outputs.last_hidden_state
        )  # (batch_size, max_len, hidden_size)

        vagueness_logits = self.vagueness_classifier(sequence_output)
        readability_logits = self.readability_classifier(sequence_output)

        return vagueness_logits, readability_logits

In [10]:
model_1_path = "group_1_model.pth"
model_2_path = "group_2_model.pth"
model_3_path = "group_3_model.pth"

In [11]:
# load models from file
model_1 = MultiLabelBERT_Political(
    bert_model_name="bert-base-uncased",
    num_stance_labels=3,
    num_group_appeals_labels=2,
    num_endorsement_labels=2,
)
model_1.load_state_dict(torch.load(model_1_path))
model_1.eval()
model_2 = MultiLabelBERT_Sentiment(
    bert_model_name="bert-base-uncased",
    num_hyperbol_labels=2,
    num_sentiment_labels=6,
)
model_2.load_state_dict(torch.load(model_2_path))
model_2.eval()
model_3 = MultiLabelBERT_Readability(
    bert_model_name="bert-base-uncased",
    num_vagueness_labels=2,
    num_readability_labels=6,
)
model_3.load_state_dict(torch.load(model_3_path))
model_3.eval()
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1.to(device)
model_2.to(device)
model_3.to(device)

C:\Users\rishi\AppData\Local\Temp\ipykernel_10908\1866976455.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_1.load_state_dict(torch.load(model_1_path))
C:\Users\r

MultiLabelBERT_Readability(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [22]:
text = "An Australian father who is believed to have been brainwashed by the preachings of Islamic extremist Musa Cerantonio has joined the ranks of terror group Islamic State to fight in the Middle East. According to Twitter, Zia Abdul Haq's current location is in an IS-controlled area of Ash-Shaam, also known as Syria. Friends of the 33-year-old believed the man from Logan - south of Brisbane - was moving to Syria to look for a new wife after the breakdown of his marriage, but The Courier Mail has revealed Abdul Haq is in fact fighting with the extremist organisation. Scroll down for video . The friends of Zia Abdul Haq says the father-of-one, from Logan, is a 'good-hearted guy' Two of Australia's most notorious jihadists Khaled Sharrouf and Mohamed Elomar are part of the same group. Abdul Haq is one of the latest of three men who are believed to have joined IS, including Melbourne's Abu Khaled al-Cambodi and Mounir Raad. The 33-year-old's friends described him as a 'good-hearted guy' but had been 'brainwashed' by Cerantonio - who is believed to be an influential IS figure - after hearing the radical sheik speak in Brisbane two years ago. The 33-year-old's friends claim he was brainwashed by prominent IS supporter, Musa Cerantonio - a radical sheik . 'He wasn’t a bad guy but he fell off the rails and under the spell of the extremists,' friend Yehya El-Kholed told The Courier Mail. 'Sheik Cerantonio is a hypocrite because while he urges people to go to Syria, he won’t go himself.' Since he landed in Syria, Abdul Haq - who came to Australia in his 20s and took up a job as a finance officer - has been actively posting photos depicting life as a fighter for IS on the social media site. These include M16 training, jihadists playing violent video games and cooking up a shared meal. But like Sharrouf and Elomar, Abdul Haq - who has a young son - has used Twitter as platform to fire off threatening messages, including one to foreign forces to 'bring dead body bag for each of your soldiers'. Most recently, he took aim at the U.S. saying 'Send you (men) to fight us, your drones are useless on the ground'  and 'How come you fight #IS with your gays & lesbians army' on Saturday night. Abdul Haq has also hit back at media outlets circulating news about him. 'Many lies about me in Australian news. =/,' he tweeted within half an hour of reports emerging. Since arriving in Syria Abdul Haq has been posting photos on Twitter of what it is like being an IS fighter . Last month Musa Cerantonio was deported back to Australia from the Philippines after he was arrested after breaching the Philippines' immigration laws. Changing his name from Robert Edward, the 29-year-old was detained in the central Filipino city of Cebu where he was believed to have been hiding from Australian authorities, despite claiming on Twitter he had arrived in the Middle East just days before his arrest. He was found with a Filipino fashion designer claiming to be his wife, inside an apartment. Australian authorities cancelled his passport, triggering his arrest and leaving without any documentation. He was then set free when he arrived in Melbourne after deportation as he had not broken any Australian laws. Musa Cerantonio is thought to be an influential IS figure and Abdul Haq is believed to have taken up extremist views after hearing the sheik speak in 2012 in Brisbane ."
inputs = tokenizer(
    text,
    return_tensors="pt",
    max_length=512,
    padding="max_length",
    truncation=True,
)
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    stance_logits, group_appeals_logits, endorsement_logits = model_1(
        inputs["input_ids"], inputs["attention_mask"]
    )
    hyperbole_logits, sentiment_logits = model_2(
        inputs["input_ids"], inputs["attention_mask"]
    )
    vagueness_logits, readability_logits = model_3(
        inputs["input_ids"], inputs["attention_mask"]
    )

    pred_labels_stance = torch.argmax(stance_logits, dim=2).squeeze().cpu().numpy()
    pred_labels_group_appeals = torch.argmax(group_appeals_logits, dim=2).squeeze().cpu().numpy()
    pred_labels_endorsement = torch.argmax(endorsement_logits, dim=2).squeeze().cpu().numpy()
    pred_labels_hyperbole = torch.argmax(hyperbole_logits, dim=2).squeeze().cpu().numpy()
    pred_labels_sentiment = torch.argmax(sentiment_logits, dim=2).squeeze().cpu().numpy()
    pred_labels_vagueness = torch.argmax(vagueness_logits, dim=2).squeeze().cpu().numpy()
    pred_labels_readability = torch.argmax(readability_logits, dim=2).squeeze().cpu().numpy()

    # get labels
    stance_labels = ["Left", "Center", "Right"]
    group_appeals_labels = ["No Group Appeal", "Group Appeal"]
    endorsement_labels = ["No Endorsement", "Endorsement"]
    hyperbole_labels = ["No Hyperbole", "Hyperbole"]
    sentiment_labels = [
        "Joy",
        "Sadness",
        "Anger",
        "Fear",
        "Love",
        "Surprise",
    ]
    vagueness_labels = ["No Vagueness", "Vagueness"]
    readability_labels = [
        "Extremely Hard",
        "Hard",
        "Normal",
        "Easy",
        "Very Easy",
        "Extremely Easy",
    ]

    # get tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().cpu().numpy())
    tokens = [token for token in tokens if token not in ["[CLS]", "[SEP]", "[PAD]"]]
    tokens = tokens[: len(pred_labels_stance)]
    pred_labels_stance = pred_labels_stance[: len(tokens)]
    pred_labels_group_appeals = pred_labels_group_appeals[: len(tokens)]
    pred_labels_endorsement = pred_labels_endorsement[: len(tokens)]
    pred_labels_hyperbole = pred_labels_hyperbole[: len(tokens)]
    pred_labels_sentiment = pred_labels_sentiment[: len(tokens)]
    pred_labels_vagueness = pred_labels_vagueness[: len(tokens)]
    pred_labels_readability = pred_labels_readability[: len(tokens)]

    # get predictions
    pred_stance = [stance_labels[label] for label in pred_labels_stance]
    pred_group_appeals = [
        group_appeals_labels[label] for label in pred_labels_group_appeals
    ]
    pred_endorsement = [
        endorsement_labels[label] for label in pred_labels_endorsement
    ]
    pred_hyperbole = [hyperbole_labels[label] for label in pred_labels_hyperbole]
    pred_sentiment = [
        sentiment_labels[label] for label in pred_labels_sentiment
    ]
    pred_vagueness = [vagueness_labels[label] for label in pred_labels_vagueness]
    pred_readability = [
        readability_labels[label] for label in pred_labels_readability
    ]

    # create json
    tokens_dict = {}
    for i, token in enumerate(tokens):
        tokens_dict[token] = {
            "token": token,
            "stance": str(pred_labels_stance[i]),
            "group_appeals": str(pred_labels_group_appeals[i]),
            "endorsement": str(pred_labels_endorsement[i]),
            "hyperbole": str(pred_labels_hyperbole[i]),
            "sentiment": str(pred_labels_sentiment[i]),
            "vagueness": str(pred_labels_vagueness[i]),
            "readability": str(pred_labels_readability[i]),
        }

    json_data = {
        "text": text,
        "tokens": tokens_dict,
    }
    json_file = "output/output.json"
    with open(json_file, "w") as f:
        json.dump(json_data, f, indent=4)
    print(f"JSON file saved to {json_file}")

JSON file saved to output/output.json


In [ ]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer,
    BertModel,
)
import json

class MultiLabelBERT_Political(torch.nn.Module):
    """BERT model for multi-label token classification."""

    def __init__(
        self,
        bert_model_name,
        num_stance_labels,
        num_group_appeals_labels,
        num_endorsement_labels,
    ):
        super(MultiLabelBERT_Political, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.stance_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_stance_labels
        )
        self.group_appeals_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_group_appeals_labels
        )
        self.endorsement_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_endorsement_labels
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = (
            outputs.last_hidden_state
        )  # (batch_size, max_len, hidden_size)

        stance_logits = self.stance_classifier(sequence_output)
        group_appeals_logits = self.group_appeals_classifier(sequence_output)
        endorsement_logits = self.endorsement_classifier(sequence_output)

        return stance_logits, group_appeals_logits, endorsement_logits


class MultiLabelBERT_Sentiment(torch.nn.Module):
    """BERT model for multi-label token classification."""

    def __init__(self, bert_model_name, num_hyperbol_labels, num_sentiment_labels):
        super(MultiLabelBERT_Sentiment, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.hyperbole_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_hyperbol_labels
        )
        self.sentiment_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_sentiment_labels
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = (
            outputs.last_hidden_state
        )  # (batch_size, max_len, hidden_size)

        hyperbole_logits = self.hyperbole_classifier(sequence_output)
        sentiment_logits = self.sentiment_classifier(sequence_output)

        return hyperbole_logits, sentiment_logits


class MultiLabelBERT_Readability(torch.nn.Module):
    """BERT model for multi-label token classification."""

    def __init__(self, bert_model_name, num_vagueness_labels, num_readability_labels):
        super(MultiLabelBERT_Readability, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.vagueness_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_vagueness_labels
        )
        self.readability_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, num_readability_labels
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = (
            outputs.last_hidden_state
        )  # (batch_size, max_len, hidden_size)

        vagueness_logits = self.vagueness_classifier(sequence_output)
        readability_logits = self.readability_classifier(sequence_output)

        return vagueness_logits, readability_logits

def main(input_text: str):
    # Load the model and tokenizer
    model_1 = MultiLabelBERT_Political(
        bert_model_name="bert-base-uncased",
        num_stance_labels=3,
        num_group_appeals_labels=2,
        num_endorsement_labels=2,
    )
    model_1.load_state_dict(torch.load(model_1_path))
    model_1.eval()
    model_2 = MultiLabelBERT_Sentiment(
        bert_model_name="bert-base-uncased",
        num_hyperbol_labels=2,
        num_sentiment_labels=6,
    )
    model_2.load_state_dict(torch.load(model_2_path))
    model_2.eval()
    model_3 = MultiLabelBERT_Readability(
        bert_model_name="bert-base-uncased",
        num_vagueness_labels=2,
        num_readability_labels=6,
    )
    model_3.load_state_dict(torch.load(model_3_path))
    model_3.eval()
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_1.to(device)
    model_2.to(device)
    model_3.to(device)

    # Load the data
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=512,
        padding="max_length",
        truncation=True,
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        stance_logits, group_appeals_logits, endorsement_logits = model_1(
            inputs["input_ids"], inputs["attention_mask"]
        )
        hyperbole_logits, sentiment_logits = model_2(
            inputs["input_ids"], inputs["attention_mask"]
        )
        vagueness_logits, readability_logits = model_3(
            inputs["input_ids"], inputs["attention_mask"]
        )

        pred_labels_stance = torch.argmax(stance_logits, dim=2).squeeze().cpu().numpy()
        pred_labels_group_appeals = torch.argmax(group_appeals_logits, dim=2).squeeze().cpu().numpy()
        pred_labels_endorsement = torch.argmax(endorsement_logits, dim=2).squeeze().cpu().numpy()
        pred_labels_hyperbole = torch.argmax(hyperbole_logits, dim=2).squeeze().cpu().numpy()
        pred_labels_sentiment = torch.argmax(sentiment_logits, dim=2).squeeze().cpu().numpy()
        pred_labels_vagueness = torch.argmax(vagueness_logits, dim=2).squeeze().cpu().numpy()
        pred_labels_readability = torch.argmax(readability_logits, dim=2).squeeze().cpu().numpy()


        # get tokens
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().cpu().numpy())
        tokens = [token for token in tokens if token not in ["[CLS]", "[SEP]", "[PAD]"]]
        tokens = tokens[: len(pred_labels_stance)]
        pred_labels_stance = pred_labels_stance[: len(tokens)]
        pred_labels_group_appeals = pred_labels_group_appeals[: len(tokens)]
        pred_labels_endorsement = pred_labels_endorsement[: len(tokens)]
        pred_labels_hyperbole = pred_labels_hyperbole[: len(tokens)]
        pred_labels_sentiment = pred_labels_sentiment[: len(tokens)]
        pred_labels_vagueness = pred_labels_vagueness[: len(tokens)]
        pred_labels_readability = pred_labels_readability[: len(tokens)]

        # # get labels
        # stance_labels = ["Left", "Center", "Right"]
        # group_appeals_labels = ["No Group Appeal", "Group Appeal"]
        # endorsement_labels = ["No Endorsement", "Endorsement"]
        # hyperbole_labels = ["No Hyperbole", "Hyperbole"]
        # sentiment_labels = [
        #     "Joy",
        #     "Sadness",
        #     "Anger",
        #     "Fear",
        #     "Love",
        #     "Surprise",
        # ]
        # vagueness_labels = ["No Vagueness", "Vagueness"]
        # readability_labels = [
        #     "Extremely Hard",
        #     "Hard",
        #     "Normal",
        #     "Easy",
        #     "Very Easy",
        #     "Extremely Easy",
        # ]

        
        # # get predictions
        # pred_stance = [stance_labels[label] for label in pred_labels_stance]
        # pred_group_appeals = [
        #     group_appeals_labels[label] for label in pred_labels_group_appeals
        # ]
        # pred_endorsement = [
        #     endorsement_labels[label] for label in pred_labels_endorsement
        # ]
        # pred_hyperbole = [hyperbole_labels[label] for label in pred_labels_hyperbole]
        # pred_sentiment = [
        #     sentiment_labels[label] for label in pred_labels_sentiment
        # ]
        # pred_vagueness = [vagueness_labels[label] for label in pred_labels_vagueness]
        # pred_readability = [
        #     readability_labels[label] for label in pred_labels_readability
        # ]

        # create json
        tokens_dict = {}
        for i, token in enumerate(tokens):
            tokens_dict[token] = {
                "token": token,
                "stance": str(pred_labels_stance[i]),
                "group_appeals": str(pred_labels_group_appeals[i]),
                "endorsement": str(pred_labels_endorsement[i]),
                "hyperbole": str(pred_labels_hyperbole[i]),
                "sentiment": str(pred_labels_sentiment[i]),
                "vagueness": str(pred_labels_vagueness[i]),
                "readability": str(pred_labels_readability[i]),
            }

        json_data = {
            "text": text,
            "tokens": tokens_dict,
        }
        json_file = "output/output.json"
        with open(json_file, "w") as f:
            json.dump(json_data, f, indent=4)
        print(f"JSON file saved to {json_file}")